I ran a transitive closure yesterday, and I am puzzled by some of the behaviour I saw.
I'm creating this notebook page `explore-fruit-fly-transitive-closure` to investigate it.

First, since much of what I want to do involves SQL queries, I'll experiment with a helper function `show`.

In [1]:
from s2ag.persistence.database_catalogue import DatabaseCatalogue, test_connection

In [2]:
connection = test_connection()
cursor = connection.cursor()
catalogue = DatabaseCatalogue(connection)

In [3]:
def show(sql_query: str, *params):
    try:
        cursor.execute(sql_query, params)
        for row in cursor.fetchall():
            print(row)
    except Exception as e:
        print('oops! %s' % e)
        connection.rollback()

In [4]:
fly_structured_id = 'e2e1aa8b8b1dfed9c65589d5293acbae4cbe061a'

In [5]:
show('select paper_id, title from paper where paper_id=(%s)', fly_structured_id)

('e2e1aa8b8b1dfed9c65589d5293acbae4cbe061a', 'Structured sampling of olfactory input by the fly mushroom body')


In [6]:
show('select citing_id from citation where cited_id=(%s)', fly_structured_id)


('3149671f907078eac6835a05a99da756c493a15b',)


In [7]:
show('select paper_id, title from paper where paper_id=(%s)', '3149671f907078eac6835a05a99da756c493a15b')

('3149671f907078eac6835a05a99da756c493a15b', 'FlyWire: Online community for whole-brain connectomics')


In [10]:
len(catalogue.paper_ids())

226

So there are 226 papers in the test database at present.

How many citations do they have?

In [13]:
papers = {}
for pid in catalogue.paper_ids():
    paper = catalogue.read_paper(pid)
    papers[pid] = paper

In [14]:
paper_list = list(papers[pid] for pid in papers.keys())

In [23]:
triply_cited_papers = list(paper for paper in paper_list if paper.citation_count  >= 3)

In [24]:
sorted_papers = sorted(triply_cited_papers, key=lambda paper: paper.citation_count)

In [25]:
for paper in sorted_papers[:20]:
    print(paper.title, paper.paper_id, paper.citation_count)

Molecular and neural mechanisms regulating sexual motivation of virgin female Drosophila d23508ee81467dff9435d42d1d4633e178e59992 3
A Survey of Visualization and Analysis in High‐Resolution Connectomics ebaf6fafb4adb7f3657fdbb869249c727ffccf7e 3
Endocrine cybernetics: neuropeptides as molecular switches in behavioural decisions 9088b5478fc706f368d2d1fd5661aba9384d782b 3
Petascale pipeline for precise alignment of images from serial section electron microscopy a641df4e738354237c2ba9cdb13c7dbca7d58171 3
Gliotransmission of D-serine promotes thirst-directed behaviors in Drosophila d798c672b1088b01ddcb747d395a35e9cca48f19 3
Mating-driven variability in olfactory local interneuron wiring 564eda3bdb4e72494a6e8bfca32acab9067bef97 3
Descending neurons coordinate anterior grooming behavior in Drosophila 53b24a8666c1c8be96a47a24e5ce1ca9b3261aaa 3
CONFIRMS: A Toolkit for Scalable, Black Box Connectome Assessment and Investigation 8d51d2e27be1df78179f8b2a19fc002f46228408 3
Niwaki Instead of Random

In [26]:
import json

with open('sample_02.json', 'w') as sample:
    paper = papers['d23508ee81467dff9435d42d1d4633e178e59992']
    sample.write(json.dumps(paper.jason_dictionary))


In [8]:
# cursor.close()
# connection.close()